In [1]:
import os
import sys
import argparse
import time

import torch.utils.model_zoo as model_zoo
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
import torch.backends.cudnn as cudnn
import torchvision
from torch.utils.tensorboard import SummaryWriter

import datasets
from model import L2CS
from utils import select_device

In [2]:
!nvidia-smi

Mon Jun 13 21:52:11 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 470.57.02    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  On   | 00000000:10:1D.0 Off |                    0 |
| N/A   47C    P0    62W / 400W |      0MiB / 40536MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA A100-SXM...  On   | 00000000:20:1C.0 Off |                    0 |
| N/A   

In [3]:
!ls /project/data/sdata/Label

p00.label  p03.label  p06.label  p09.label  p12.label
p01.label  p04.label  p07.label  p10.label  p13.label
p02.label  p05.label  p08.label  p11.label  p14.label


In [5]:
!ls /project/data/sdata/Image/face

fold00	fold02	fold04	fold06	fold08	fold10	fold12	fold14
fold01	fold03	fold05	fold07	fold09	fold11	fold13


In [6]:
def parse_args():
    """Parse input arguments."""
    parser = argparse.ArgumentParser(description='Gaze estimation using L2CSNet.')
#     # Gaze360
#     parser.add_argument(
#         '--gaze360image_dir', dest='gaze360image_dir', help='Directory path for gaze images.',
#         default='datasets/Gaze360/Image', type=str)
#     parser.add_argument(
#         '--gaze360label_dir', dest='gaze360label_dir', help='Directory path for gaze labels.',
#         default='datasets/Gaze360/Label/train.label', type=str)
#     # mpiigaze
    parser.add_argument(
        '--gazeMpiimage_dir', dest='gazeMpiimage_dir', help='Directory path for gaze images.',
        default='/project/data/Image', type=str)
    parser.add_argument(
        '--gazeMpiilabel_dir', dest='gazeMpiilabel_dir', help='Directory path for gaze labels.',
        default='/project/data/Label', type=str)

    # Important args -------------------------------------------------------------------------------------------------------
    # ----------------------------------------------------------------------------------------------------------------------
    parser.add_argument(
        '--dataset', dest='dataset', help='mpiigaze, rtgene, gaze360, ethgaze',
        default= "gaze360", type=str)
    parser.add_argument(
        '--output', dest='output', help='Path of output models.',
        default='output/snapshots/', type=str)
    parser.add_argument(
        '--snapshot', dest='snapshot', help='Path of model snapshot.',
        default='', type=str)
    parser.add_argument(
        '--gpu', dest='gpu_id', help='GPU device id to use [0] or multiple 0,1,2,3',
        default='0', type=str)
    parser.add_argument(
        '--num_epochs', dest='num_epochs', help='Maximum number of training epochs.',
        default=60, type=int)
    parser.add_argument(
        '--batch_size', dest='batch_size', help='Batch size.',
        default=1, type=int)
    parser.add_argument(
        '--arch', dest='arch', help='Network architecture, can be: ResNet18, ResNet34, [ResNet50], ''ResNet101, ResNet152, Squeezenet_1_0, Squeezenet_1_1, MobileNetV2',
        default='ResNet50', type=str)
    parser.add_argument(
        '--alpha', dest='alpha', help='Regression loss coefficient.',
        default=1, type=float)
    parser.add_argument(
        '--lr', dest='lr', help='Base learning rate.',
        default=0.00001, type=float)
    # ---------------------------------------------------------------------------------------------------------------------
    # Important args ------------------------------------------------------------------------------------------------------
    args = parser.parse_args()
    return args

In [7]:
!ls /project/results

output	soutput


In [8]:
class Nothing:
    pass
args = Nothing()
args.gazeMpiimage_dir = '/project/data/sdata/Image'
args.gazeMpiilabel_dir = '/project/data/sdata/Label'
args.output = '/project/results/soutput/snapshots/'
args.dataset = 'mpiigaze'
args.snapshot=''
args.gpu_id = '0,1,2,3'
args.num_epochs = 60
args.batch_size = 100
args.arch = 'ResNet50'
args.alpha = 1.0
args.lr = 0.00001



In [9]:
def get_ignored_params(model):
    # Generator function that yields ignored params.
    b = [model.conv1, model.bn1, model.fc_finetune]
    for i in range(len(b)):
        for module_name, module in b[i].named_modules():
            if 'bn' in module_name:
                module.eval()
            for name, param in module.named_parameters():
                yield param

def get_non_ignored_params(model):
    # Generator function that yields params that will be optimized.
    b = [model.layer1, model.layer2, model.layer3, model.layer4]
    for i in range(len(b)):
        for module_name, module in b[i].named_modules():
            if 'bn' in module_name:
                module.eval()
            for name, param in module.named_parameters():
                yield param

def get_fc_params(model):
    # Generator function that yields fc layer params.
    b = [model.fc_yaw_gaze, model.fc_pitch_gaze]
    for i in range(len(b)):
        for module_name, module in b[i].named_modules():
            for name, param in module.named_parameters():
                yield param
                
def load_filtered_state_dict(model, snapshot):
    # By user apaszke from discuss.pytorch.org
    model_dict = model.state_dict()
    snapshot = {k: v for k, v in snapshot.items() if k in model_dict}
    model_dict.update(snapshot)
    model.load_state_dict(model_dict)


def getArch_weights(arch, bins):
    if arch == 'ResNet18':
        model = L2CS(torchvision.models.resnet.BasicBlock, [2, 2, 2, 2], bins)
        pre_url = 'https://download.pytorch.org/models/resnet18-5c106cde.pth'
    elif arch == 'ResNet34':
        model = L2CS(torchvision.models.resnet.BasicBlock, [3, 4, 6, 3], bins)
        pre_url = 'https://download.pytorch.org/models/resnet34-333f7ec4.pth'
    elif arch == 'ResNet101':
        model = L2CS(torchvision.models.resnet.Bottleneck, [3, 4, 23, 3], bins)
        pre_url = 'https://download.pytorch.org/models/resnet101-5d3b4d8f.pth'
    elif arch == 'ResNet152':
        model = L2CS(torchvision.models.resnet.Bottleneck,[3, 8, 36, 3], bins)
        pre_url = 'https://download.pytorch.org/models/resnet152-b121ed2d.pth'
    else:
        if arch != 'ResNet50':
            print('Invalid value for architecture is passed! '
                  'The default value of ResNet50 will be used instead!')
        model = L2CS(torchvision.models.resnet.Bottleneck, [3, 4, 6, 3], bins)
        pre_url = 'https://download.pytorch.org/models/resnet50-19c8e357.pth'

    return model, pre_url

In [11]:
# args = parse_args()
cudnn.enabled = True
num_epochs = args.num_epochs
batch_size = args.batch_size
data_set=args.dataset
alpha = args.alpha
output=args.output

os.makedirs(output, exist_ok=True)

In [12]:
gpu = select_device(args.gpu_id, batch_size=args.batch_size)
print(gpu)

cuda:0


In [13]:
transformations = transforms.Compose([
    transforms.Resize(448),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

In [15]:
#for tensorboard graphs
folder = os.listdir(args.gazeMpiilabel_dir)
folder.sort()
print(folder)

['p00.label', 'p01.label', 'p02.label', 'p03.label', 'p04.label', 'p05.label', 'p06.label', 'p07.label', 'p08.label', 'p09.label', 'p10.label', 'p11.label', 'p12.label', 'p13.label', 'p14.label']


In [16]:
testlabelpathcombined = [os.path.join(args.gazeMpiilabel_dir, j) for j in folder]
fold = 0

dataset=datasets.Mpiigaze(testlabelpathcombined,args.gazeMpiimage_dir, transformations, True, 180, fold)
train_loader_gaze = DataLoader(
    dataset=dataset,
    batch_size=int(batch_size),
    shuffle=True,
    num_workers=4,
    pin_memory=True)
examples = iter(train_loader_gaze)
example_images,labels_gaze, cont_labels_gaze, name = examples.next()

0 items removed from dataset that have an angle > 180


In [17]:
%%time
start = time.time()

folder = os.listdir(args.gazeMpiilabel_dir)
folder.sort()
testlabelpathcombined = [os.path.join(args.gazeMpiilabel_dir, j) for j in folder]
for fold in range(15):
    model, pre_url = getArch_weights(args.arch, 28)
    print(fold, model.conv1)
    load_filtered_state_dict(model, model_zoo.load_url(pre_url))
#     model = nn.DataParallel(model)
#     model.to(gpu)
#     print('Again, model.conv1', model.conv1)
    print('Loading data.')
    dataset=datasets.Mpiigaze(testlabelpathcombined,args.gazeMpiimage_dir, transformations, True, 180, fold)
    
    train_loader_gaze = DataLoader(
        dataset=dataset,
        batch_size=int(batch_size),
        shuffle=True,
        num_workers=4,
        pin_memory=True)
    
    torch.backends.cudnn.benchmark = True
    
    fold_path = os.path.join(output, 'fold' + f'{fold:0>2}'+'/')
    print(f"fold_path is {fold_path}")
    if not os.path.exists(fold_path):
        os.makedirs(fold_path)
        
    writer =SummaryWriter(fold_path)
    
    
    example_images = Variable(example_images)
    writer.add_graph(model, example_images)
        
#     summary_name = '{}_{}'.format('L2CS-mpiigaze', int(time.time()))
# #     print('summary_name=', summary_name)

#     if not os.path.exists(os.path.join(output+'/{}'.format(summary_name),'fold' + str(fold))):
#         os.makedirs(os.path.join(output+'/{}'.format(summary_name),'fold' + str(fold)))

    
    criterion = nn.CrossEntropyLoss().cuda(gpu)
    reg_criterion = nn.MSELoss().cuda(gpu)
    softmax = nn.Softmax(dim=1).cuda(gpu)
    idx_tensor = [idx for idx in range(28)]
    idx_tensor = Variable(torch.FloatTensor(idx_tensor)).cuda(gpu)

#     print('Again, model.conv1', model.conv1)
    # Optimizer gaze
    #### origianally wrong number of arguments
    optimizer_gaze = torch.optim.Adam([
        {'params': get_ignored_params(model), 'lr': 0}, 
        {'params': get_non_ignored_params(model), 'lr': args.lr},
        {'params': get_fc_params(model), 'lr': args.lr}
    ], args.lr)

    
    
    configuration = f"\ntrain configuration, gpu_id={args.gpu_id}, batch_size={batch_size}, model_arch={args.arch}\n Start training dataset={data_set}, loader={len(train_loader_gaze)}, fold={fold}--------------\n"
#     print(configuration)
    model.to(gpu)
    model = nn.DataParallel(model, device_ids=[0,1,2,3])
    
    for epoch in range(num_epochs):
        sum_loss_pitch_gaze = sum_loss_yaw_gaze = iter_gaze = 0

        for i, (images_gaze, labels_gaze, cont_labels_gaze,name) in enumerate(train_loader_gaze):
            images_gaze = Variable(images_gaze).cuda(gpu)

            # Binned labels
            label_pitch_gaze = Variable(labels_gaze[:, 0]).cuda(gpu)
            label_yaw_gaze = Variable(labels_gaze[:, 1]).cuda(gpu)

            # Continuous labels
            label_pitch_cont_gaze = Variable(cont_labels_gaze[:, 0]).cuda(gpu)
            label_yaw_cont_gaze = Variable(cont_labels_gaze[:, 1]).cuda(gpu)

            pitch, yaw = model(images_gaze)

            # Cross entropy loss
            loss_pitch_gaze = criterion(pitch, label_pitch_gaze)
            loss_yaw_gaze = criterion(yaw, label_yaw_gaze)

            # MSE loss
            pitch_predicted = softmax(pitch)
            yaw_predicted = softmax(yaw)

            # mapping from binned (0 to 28) to angels (-42 to 42) 
            pitch_predicted = \
                torch.sum(pitch_predicted * idx_tensor, 1) * 3 - 42
            yaw_predicted = \
                torch.sum(yaw_predicted * idx_tensor, 1) * 3 - 42

            loss_reg_pitch = reg_criterion(
                pitch_predicted, label_pitch_cont_gaze)
            loss_reg_yaw = reg_criterion(
                yaw_predicted, label_yaw_cont_gaze)

            
            # Total loss
            loss_pitch_gaze += alpha * loss_reg_pitch
            loss_yaw_gaze += alpha * loss_reg_yaw

            sum_loss_pitch_gaze += loss_pitch_gaze
            sum_loss_yaw_gaze += loss_yaw_gaze

            
            
            loss_seq = [loss_pitch_gaze, loss_yaw_gaze]
            grad_seq = \
                [torch.tensor(1.0).cuda(gpu) for _ in range(len(loss_seq))]

            optimizer_gaze.zero_grad(set_to_none=True)
            torch.autograd.backward(loss_seq, grad_seq)
            optimizer_gaze.step()

            iter_gaze += 1
            
            iterations = len(dataset)//batch_size
            div10 = iterations/10
            if (i+1) % div10 == 0:  #for every div10 batches
                now=time.time()
                elapsed = now-start
                
                yaw_loss = sum_loss_pitch_gaze/iter_gaze
                pitch_loss = sum_loss_yaw_gaze/iter_gaze
                print(f'Fold: {fold} Epoch [{epoch+1}/{num_epochs}], Iter [{i+1}/{len(dataset)//batch_size}] Losses: '
                        f'Gaze Yaw {yaw_loss:.4f},Gaze Pitch {pitch_loss:.3f}'
                         f' elapsed:{elapsed:.1f}s')
                


        # Save models at numbered epochs.
#         print("writing to tensor board...")
        writer.add_scalar('pitch_loss', pitch_loss, epoch)
        writer.add_scalar('yaw_loss', yaw_loss, epoch)
#         wandb.log(a dictinary)
        
        if epoch % 1 == 0 and epoch < num_epochs:
            print(f"fold_path is still? {fold_path}")
            pathf = fold_path + 'epoch_' + str(epoch+1) + '.pkl'
            print(pathf)
            print('Taking snapshot...')
            
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer_gaze
                    .state_dict(),
                'pitch_loss': pitch_loss,
                'yaw_loss': yaw_loss
                }, pathf)
            
    writer.close()   #after each fold
            
#             torch.save(model.state_dict(), pathf) 
#                             output+'/fold' + str(fold) +'/'+
#                             'epoch_' + str(epoch+1) + '.pkl')
#                 )

      

0 Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)


Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/hub/checkpoints/resnet50-19c8e357.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

Loading data.
0 items removed from dataset that have an angle > 180
fold_path is /project/results/soutput/snapshots/fold00/


/usr/local/lib/python3.8/dist-packages/torch/optim/adam.py:81: UserWarning: optimizer contains a parameter group with duplicate parameters; in future, this will cause an error; see github.com/pytorch/pytorch/issues/40967 for more information
  super(Adam, self).__init__(params, defaults)


...

Fold: 14 Epoch [25/60], Iter [168/420] Losses: Gaze Yaw 1.8128,Gaze Pitch 1.281 elapsed:81179.5s
Fold: 14 Epoch [25/60], Iter [210/420] Losses: Gaze Yaw 1.6953,Gaze Pitch 1.292 elapsed:81190.3s
Fold: 14 Epoch [25/60], Iter [252/420] Losses: Gaze Yaw 1.6331,Gaze Pitch 1.263 elapsed:81200.9s
Fold: 14 Epoch [25/60], Iter [294/420] Losses: Gaze Yaw 1.5757,Gaze Pitch 1.239 elapsed:81211.4s
Fold: 14 Epoch [25/60], Iter [336/420] Losses: Gaze Yaw 1.5527,Gaze Pitch 1.246 elapsed:81222.3s
Fold: 14 Epoch [25/60], Iter [378/420] Losses: Gaze Yaw 1.5547,Gaze Pitch 1.236 elapsed:81232.9s
Fold: 14 Epoch [25/60], Iter [420/420] Losses: Gaze Yaw 1.5250,Gaze Pitch 1.242 elapsed:81243.5s
fold_path is still? /project/results/soutput/snapshots/fold14/
/project/results/soutput/snapshots/fold14/epoch_25.pkl
Taking snapshot...
Fold: 14 Epoch [26/60], Iter [42/420] Losses: Gaze Yaw 1.4407,Gaze Pitch 1.288 elapsed:81256.7s
Fold: 14 Epoch [26/60], Iter [84/420] Losses: Gaze Yaw 1.4022,Gaze Pitch 1.235 elapsed:8

Fold: 14 Epoch [32/60], Iter [378/420] Losses: Gaze Yaw 1.2142,Gaze Pitch 1.091 elapsed:81887.0s
Fold: 14 Epoch [32/60], Iter [420/420] Losses: Gaze Yaw 1.2229,Gaze Pitch 1.123 elapsed:81895.0s
fold_path is still? /project/results/soutput/snapshots/fold14/
/project/results/soutput/snapshots/fold14/epoch_32.pkl
Taking snapshot...
Fold: 14 Epoch [33/60], Iter [42/420] Losses: Gaze Yaw 1.4335,Gaze Pitch 0.938 elapsed:81908.0s
Fold: 14 Epoch [33/60], Iter [84/420] Losses: Gaze Yaw 1.5488,Gaze Pitch 1.025 elapsed:81918.5s
Fold: 14 Epoch [33/60], Iter [126/420] Losses: Gaze Yaw 1.5530,Gaze Pitch 1.048 elapsed:81929.2s
Fold: 14 Epoch [33/60], Iter [168/420] Losses: Gaze Yaw 1.4220,Gaze Pitch 1.021 elapsed:81940.2s
Fold: 14 Epoch [33/60], Iter [210/420] Losses: Gaze Yaw 1.3291,Gaze Pitch 1.014 elapsed:81950.8s
Fold: 14 Epoch [33/60], Iter [252/420] Losses: Gaze Yaw 1.2804,Gaze Pitch 1.021 elapsed:81961.6s
Fold: 14 Epoch [33/60], Iter [294/420] Losses: Gaze Yaw 1.2515,Gaze Pitch 1.018 elapsed:8

Fold: 14 Epoch [40/60], Iter [126/420] Losses: Gaze Yaw 1.0513,Gaze Pitch 0.918 elapsed:82553.9s
Fold: 14 Epoch [40/60], Iter [168/420] Losses: Gaze Yaw 1.0820,Gaze Pitch 0.912 elapsed:82561.8s
Fold: 14 Epoch [40/60], Iter [210/420] Losses: Gaze Yaw 1.0794,Gaze Pitch 0.931 elapsed:82569.7s
Fold: 14 Epoch [40/60], Iter [252/420] Losses: Gaze Yaw 1.0919,Gaze Pitch 0.929 elapsed:82577.8s
Fold: 14 Epoch [40/60], Iter [294/420] Losses: Gaze Yaw 1.0643,Gaze Pitch 0.947 elapsed:82585.9s
Fold: 14 Epoch [40/60], Iter [336/420] Losses: Gaze Yaw 1.0564,Gaze Pitch 0.936 elapsed:82594.1s
Fold: 14 Epoch [40/60], Iter [378/420] Losses: Gaze Yaw 1.0384,Gaze Pitch 0.935 elapsed:82602.3s
Fold: 14 Epoch [40/60], Iter [420/420] Losses: Gaze Yaw 1.0200,Gaze Pitch 0.931 elapsed:82610.3s
fold_path is still? /project/results/soutput/snapshots/fold14/
/project/results/soutput/snapshots/fold14/epoch_40.pkl
Taking snapshot...
Fold: 14 Epoch [41/60], Iter [42/420] Losses: Gaze Yaw 0.9929,Gaze Pitch 0.992 elapsed:

Fold: 14 Epoch [47/60], Iter [336/420] Losses: Gaze Yaw 0.9317,Gaze Pitch 0.783 elapsed:83241.9s
Fold: 14 Epoch [47/60], Iter [378/420] Losses: Gaze Yaw 0.9323,Gaze Pitch 0.778 elapsed:83250.4s
Fold: 14 Epoch [47/60], Iter [420/420] Losses: Gaze Yaw 0.9328,Gaze Pitch 0.768 elapsed:83258.8s
fold_path is still? /project/results/soutput/snapshots/fold14/
/project/results/soutput/snapshots/fold14/epoch_47.pkl
Taking snapshot...
Fold: 14 Epoch [48/60], Iter [42/420] Losses: Gaze Yaw 0.9284,Gaze Pitch 0.747 elapsed:83269.9s
Fold: 14 Epoch [48/60], Iter [84/420] Losses: Gaze Yaw 1.0103,Gaze Pitch 0.790 elapsed:83278.6s
Fold: 14 Epoch [48/60], Iter [126/420] Losses: Gaze Yaw 1.0456,Gaze Pitch 0.893 elapsed:83286.9s
Fold: 14 Epoch [48/60], Iter [168/420] Losses: Gaze Yaw 1.0648,Gaze Pitch 0.899 elapsed:83295.4s
Fold: 14 Epoch [48/60], Iter [210/420] Losses: Gaze Yaw 1.0501,Gaze Pitch 0.863 elapsed:83304.0s
Fold: 14 Epoch [48/60], Iter [252/420] Losses: Gaze Yaw 1.0280,Gaze Pitch 0.843 elapsed:8

Fold: 14 Epoch [55/60], Iter [84/420] Losses: Gaze Yaw 0.9427,Gaze Pitch 0.730 elapsed:83886.0s
Fold: 14 Epoch [55/60], Iter [126/420] Losses: Gaze Yaw 0.9004,Gaze Pitch 0.775 elapsed:83894.5s
Fold: 14 Epoch [55/60], Iter [168/420] Losses: Gaze Yaw 0.8775,Gaze Pitch 0.737 elapsed:83902.8s
Fold: 14 Epoch [55/60], Iter [210/420] Losses: Gaze Yaw 0.8530,Gaze Pitch 0.716 elapsed:83911.3s
Fold: 14 Epoch [55/60], Iter [252/420] Losses: Gaze Yaw 0.8405,Gaze Pitch 0.694 elapsed:83919.6s
Fold: 14 Epoch [55/60], Iter [294/420] Losses: Gaze Yaw 0.8232,Gaze Pitch 0.690 elapsed:83928.1s
Fold: 14 Epoch [55/60], Iter [336/420] Losses: Gaze Yaw 0.8279,Gaze Pitch 0.697 elapsed:83936.4s
Fold: 14 Epoch [55/60], Iter [378/420] Losses: Gaze Yaw 0.8288,Gaze Pitch 0.713 elapsed:83944.9s
Fold: 14 Epoch [55/60], Iter [420/420] Losses: Gaze Yaw 0.8259,Gaze Pitch 0.712 elapsed:83953.2s
fold_path is still? /project/results/soutput/snapshots/fold14/
/project/results/soutput/snapshots/fold14/epoch_55.pkl
Taking sna

In [19]:
!ls -l /project/results/soutput

total 4
drwxr-xr-x 17 root root 6144 Jun 14 19:49 snapshots


In [ ]:
sys.exit()

In [ ]:
!nvidia-smi

In [ ]:
!ls

In [ ]:
!pwd